In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# Load and preprocess data
data = pd.read_csv("RF_imputation_NEW.csv")
data.drop(columns=['deathtime', 'survival_time', 'LOS', 'Unnamed_0', 'V1', 'admittime', 'ID', 'group', 'tLOS', 'subject_id'], inplace=True)
data['outcome'] = data['outcome'].astype(int)
predictor_names = data.columns.difference(['outcome'])

# Function to preprocess data
def preprocess_data(data, predictor_names, random_seed):
    scaler = StandardScaler()
    data[predictor_names] = scaler.fit_transform(data[predictor_names])
    train_data, valid_data = train_test_split(data, test_size=0.3, random_state=random_seed)
    return train_data, valid_data

# Preprocess the data
train_data, valid_data = preprocess_data(data, predictor_names, random_seed=42)

X_train = train_data[predictor_names]
y_train = train_data['outcome']
X_valid = valid_data[predictor_names]
y_valid = valid_data['outcome']

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Define function to create the model
def create_model(learning_rate=0.001, dropout_rate=0.5, regularization=0.001):
    model = Sequential()
    model.add(Input(shape=(X_train_resampled.shape[1],)))
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Hyperparameter tuning
best_auc = 0
best_params = {}

learning_rates = [0.001, 0.01, 0.1]
dropout_rates = [0.3, 0.5, 0.7]
regularizations = [0.001, 0.01, 0.1]
epochs_list = [50, 100]
batch_sizes = [16, 32, 64]

for lr in learning_rates:
    for dr in dropout_rates:
        for reg in regularizations:
            for epochs in epochs_list:
                for batch_size in batch_sizes:
                    print(f"Training model with lr={lr}, dr={dr}, reg={reg}, epochs={epochs}, batch_size={batch_size}")
                    model = create_model(learning_rate=lr, dropout_rate=dr, regularization=reg)
                    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
                    history = model.fit(X_train_resampled, y_train_resampled, validation_data=(X_valid, y_valid), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=0)
                    
                    y_pred_proba = model.predict(X_valid).ravel()
                    auc = roc_auc_score(y_valid, y_pred_proba)
                    print(f"AUC: {auc}")
                    
                    if auc > best_auc:
                        best_auc = auc
                        best_params = {
                            'learning_rate': lr,
                            'dropout_rate': dr,
                            'regularization': reg,
                            'epochs': epochs,
                            'batch_size': batch_size
                        }

print("Best AUC found: ", best_auc)
print("Best parameters found: ", best_params)

# Train the best model
model = create_model(learning_rate=best_params['learning_rate'], dropout_rate=best_params['dropout_rate'], regularization=best_params['regularization'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train_resampled, y_train_resampled, validation_data=(X_valid, y_valid), epochs=best_params['epochs'], batch_size=best_params['batch_size'], callbacks=[early_stopping])

# Evaluate the best model
y_pred_proba = model.predict(X_valid).ravel()
y_pred = (y_pred_proba > 0.5).astype(int)

print(classification_report(y_valid, y_pred, zero_division=0))

# Calculate AUC for the best model
auc_best = roc_auc_score(y_valid, y_pred_proba)
print(f"AUC for the best model: {auc_best}")

# Plot ROC curve for the best model
fpr_best, tpr_best, thresholds_best = roc_curve(y_valid, y_pred_proba)
plt.figure()
plt.plot(fpr_best, tpr_best, color='darkorange', lw=2, label=f'ROC curve (area = {auc_best:0.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Best DNN Model')
plt.legend(loc="lower right")
plt.show()


ModuleNotFoundError: No module named 'tensorflow'

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# Load and preprocess data
data = pd.read_csv("RF_imputation_NEW.csv")
data.drop(columns=['deathtime', 'survival_time', 'LOS', 'Unnamed_0', 'V1', 'admittime', 'ID', 'group', 'tLOS', 'subject_id'], inplace=True)
data['outcome'] = data['outcome'].astype(int)
predictor_names = data.columns.difference(['outcome'])

# Function to preprocess data
def preprocess_data(data, predictor_names, random_seed):
    scaler = StandardScaler()
    data[predictor_names] = scaler.fit_transform(data[predictor_names])
    train_data, valid_data = train_test_split(data, test_size=0.3, random_state=random_seed)
    return train_data, valid_data

# Preprocess the data
train_data, valid_data = preprocess_data(data, predictor_names, random_seed=42)

X_train = train_data[predictor_names]
y_train = train_data['outcome']
X_valid = valid_data[predictor_names]
y_valid = valid_data['outcome']

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Define the Residual Block
def residual_block(x, units, dropout_rate=0.5, regularization=0.001):
    shortcut = x
    x = Dense(units, activation='relu', kernel_regularizer=l2(regularization))(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(units, activation=None, kernel_regularizer=l2(regularization))(x)
    x = BatchNormalization()(x)
    x = Add()([shortcut, x])
    x = Dropout(dropout_rate)(x)
    x = Dense(units, activation='relu')(x)
    return x

# Define the ResNet model
def create_resnet_model(learning_rate=0.001, dropout_rate=0.5, regularization=0.001):
    inputs = Input(shape=(X_train_resampled.shape[1],))
    x = Dense(128, activation='relu', kernel_regularizer=l2(regularization))(inputs)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    
    for _ in range(3):
        x = residual_block(x, 128, dropout_rate, regularization)
    
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Hyperparameter tuning
best_auc = 0
best_params = {}

learning_rates = [0.001, 0.01, 0.1]
dropout_rates = [0.3, 0.5, 0.7]
regularizations = [0.001, 0.01, 0.1]
epochs_list = [50, 100]
batch_sizes = [16, 32, 64]

for lr in learning_rates:
    for dr in dropout_rates:
        for reg in regularizations:
            for epochs in epochs_list:
                for batch_size in batch_sizes:
                    print(f"Training model with lr={lr}, dr={dr}, reg={reg}, epochs={epochs}, batch_size={batch_size}")
                    model = create_resnet_model(learning_rate=lr, dropout_rate=dr, regularization=reg)
                    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
                    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001, verbose=1)
                    history = model.fit(X_train_resampled, y_train_resampled, validation_data=(X_valid, y_valid), 
                                        epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, reduce_lr], verbose=0)
                    
                    y_pred_proba = model.predict(X_valid).ravel()
                    auc = roc_auc_score(y_valid, y_pred_proba)
                    print(f"AUC: {auc}")
                    
                    if auc > best_auc:
                        best_auc = auc
                        best_params = {
                            'learning_rate': lr,
                            'dropout_rate': dr,
                            'regularization': reg,
                            'epochs': epochs,
                            'batch_size': batch_size
                        }

print("Best AUC found: ", best_auc)
print("Best parameters found: ", best_params)

# Train the best model
model = create_resnet_model(learning_rate=best_params['learning_rate'], 
                            dropout_rate=best_params['dropout_rate'], 
                            regularization=best_params['regularization'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001, verbose=1)
history = model.fit(X_train_resampled, y_train_resampled, validation_data=(X_valid, y_valid), 
                    epochs=best_params['epochs'], batch_size=best_params['batch_size'], callbacks=[early_stopping, reduce_lr])

# Evaluate the best model
y_pred_proba_final = model.predict(X_valid).ravel()
y_pred_final = (y_pred_proba_final > 0.5).astype(int)

print(classification_report(y_valid, y_pred_final, zero_division=0))

# Calculate AUC for the final model
auc_final = roc_auc_score(y_valid, y_pred_proba_final)
print(f"Final AUC: {auc_final}")

# Plot ROC curve for the final model
fpr_final, tpr_final, thresholds_final = roc_curve(y_valid, y_pred_proba_final)
plt.figure()
plt.plot(fpr_final, tpr_final, color='darkorange', lw=2, label=f'ROC curve (area = {auc_final:0.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Final ResNet Model')
plt.legend(loc="lower right")
plt.show()


Training model with lr=0.001, dr=0.3, reg=0.001, epochs=50, batch_size=16

Epoch 32: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 40: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
AUC: 0.7564711150918048
Training model with lr=0.001, dr=0.3, reg=0.001, epochs=50, batch_size=32

Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 26: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
AUC: 0.7775190326914465
Training model with lr=0.001, dr=0.3, reg=0.001, epochs=50, batch_size=64

Epoch 30: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 35: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
AUC: 0.7739364084191671
Training model with lr=0.001, dr=0.3, reg=0.001, epochs=100, batch_size=16

Epoch 35: ReduceLROnPlateau reducing learning 

KeyboardInterrupt: 

### new CNN-LSTM

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv1D, Flatten, MaxPooling1D, Input, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import keras_tuner as kt

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load and preprocess data
data = pd.read_csv("RF_imputation_NEW.csv")
data.drop(columns=['deathtime', 'survival_time', 'LOS', 'Unnamed_0', 'V1', 'admittime', 'ID', 'group', 'tLOS', 'subject_id'], inplace=True)
data['outcome'] = data['outcome'].astype(int)
predictor_names = data.columns.difference(['outcome'])

# Function to preprocess data
def preprocess_data(data, predictor_names, random_seed):
    scaler = StandardScaler()
    data[predictor_names] = scaler.fit_transform(data[predictor_names])
    train_data, valid_data = train_test_split(data, test_size=0.3, random_state=random_seed)
    return train_data, valid_data

# Preprocess the data
train_data, valid_data = preprocess_data(data, predictor_names, random_seed=42)

X_train = train_data[predictor_names].values
y_train = train_data['outcome'].values
X_valid = valid_data[predictor_names].values
y_valid = valid_data['outcome'].values

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Reshape data for Conv1D and LSTM
X_train_resampled = X_train_resampled.reshape((X_train_resampled.shape[0], X_train_resampled.shape[1], 1))
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1], 1))

# Define the CNN-LSTM model
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train_resampled.shape[1], 1)))
    
    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(Conv1D(filters=hp.Int(f'conv_{i}_filters', 32, 128, step=32), 
                         kernel_size=hp.Int(f'conv_{i}_kernel_size', 3, 5), 
                         activation='relu', kernel_regularizer=l2(hp.Float(f'conv_{i}_l2', 0.001, 0.01, sampling='log'))))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=2))
        model.add(Dropout(hp.Float(f'conv_{i}_dropout', 0.3, 0.7, step=0.1)))
    
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(hp.Float('lstm_dropout', 0.3, 0.7, step=0.1)))
    model.add(LSTM(64))
    model.add(Dropout(hp.Float('lstm_dropout', 0.3, 0.7, step=0.1)))
    
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(Dense(units=hp.Int(f'dense_{i}_units', 64, 256, step=64), 
                        activation='relu', kernel_regularizer=l2(hp.Float(f'dense_{i}_l2', 0.001, 0.01, sampling='log'))))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float(f'dense_{i}_dropout', 0.3, 0.7, step=0.1)))
    
    model.add(Dense(1, activation='sigmoid'))
    
    learning_rate = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='cnn_lstm_tuning'
)

# Perform hyperparameter tuning
tuner.search(
    X_train_resampled, y_train_resampled, 
    epochs=50, 
    validation_data=(X_valid, y_valid),
    callbacks=[EarlyStopping(monitor='val_loss', patience=10), 
               ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)]
)

# Get the optimal hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]
print("Best Hyperparameters:", best_hp.values)

# Train the final model with the best hyperparameters
model = tuner.hypermodel.build(best_hp)
history = model.fit(
    X_train_resampled, y_train_resampled, 
    epochs=50, 
    validation_data=(X_valid, y_valid),
    callbacks=[EarlyStopping(monitor='val_loss', patience=10), 
               ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)]
)

# Evaluate the final model
y_pred_proba_final = model.predict(X_valid).ravel()
y_pred_final = (y_pred_proba_final > 0.5).astype(int)

print(classification_report(y_valid, y_pred_final, zero_division=0))

# Calculate AUC for the final model
auc_final = roc_auc_score(y_valid, y_pred_proba_final)
print(f"Final AUC: {auc_final}")

# Plot ROC curve for the final model
fpr_final, tpr_final, thresholds_final = roc_curve(y_valid, y_pred_proba_final)
plt.figure()
plt.plot(fpr_final, tpr_final, color='darkorange', lw=2, label=f'ROC curve (area = {auc_final:0.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Final CNN-LSTM Model')
plt.legend(loc="lower right")
plt.show()



ModuleNotFoundError: No module named 'tensorflow'

### Keras CNN

In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv1D, Flatten, MaxPooling1D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import keras_tuner as kt

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load and preprocess data
data = pd.read_csv("RF_imputation_NEW.csv")
data.drop(columns=['deathtime', 'survival_time', 'LOS', 'Unnamed_0', 'V1', 'admittime', 'ID', 'group', 'tLOS', 'subject_id'], inplace=True)
data['outcome'] = data['outcome'].astype(int)
predictor_names = data.columns.difference(['outcome'])

# Function to preprocess data
def preprocess_data(data, predictor_names, random_seed):
    scaler = StandardScaler()
    data[predictor_names] = scaler.fit_transform(data[predictor_names])
    train_data, valid_data = train_test_split(data, test_size=0.3, random_state=random_seed)
    return train_data, valid_data

# Preprocess the data
train_data, valid_data = preprocess_data(data, predictor_names, random_seed=42)

X_train = train_data[predictor_names].values
y_train = train_data['outcome'].values
X_valid = valid_data[predictor_names].values
y_valid = valid_data['outcome'].values

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Reshape data for Conv1D
X_train_resampled = X_train_resampled.reshape((X_train_resampled.shape[0], X_train_resampled.shape[1], 1))
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1], 1))

# Define the model building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train_resampled.shape[1], 1)))
    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(Conv1D(filters=hp.Int(f'conv_{i}_filters', 32, 128, step=32), 
                         kernel_size=hp.Int(f'conv_{i}_kernel_size', 3, 5), 
                         activation='relu', kernel_regularizer=l2(hp.Float(f'conv_{i}_l2', 0.001, 0.01, sampling='log'))))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=2))
        model.add(Dropout(hp.Float(f'conv_{i}_dropout', 0.3, 0.7, step=0.1)))
    
    model.add(Flatten())
    
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(Dense(units=hp.Int(f'dense_{i}_units', 64, 256, step=64), 
                        activation='relu', kernel_regularizer=l2(hp.Float(f'dense_{i}_l2', 0.001, 0.01, sampling='log'))))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float(f'dense_{i}_dropout', 0.3, 0.7, step=0.1)))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')), 
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,
                        executions_per_trial=1,
                        directory='my_dir',
                        project_name='cnn_tuning')

# Perform hyperparameter tuning
tuner.search(X_train_resampled, y_train_resampled, epochs=50, validation_data=(X_valid, y_valid),
             callbacks=[EarlyStopping(monitor='val_loss', patience=10), 
                        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)])

# Get the optimal hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]
print("Best Hyperparameters:", best_hp.values)

# Train the final model with the best hyperparameters
model = tuner.hypermodel.build(best_hp)
history = model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_data=(X_valid, y_valid),
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10), 
                               ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)])

# Evaluate the final model
y_pred_proba_final = model.predict(X_valid).ravel()
y_pred_final = (y_pred_proba_final > 0.5).astype(int)

print(classification_report(y_valid, y_pred_final, zero_division=0))

# Calculate AUC for the final model
auc_final = roc_auc_score(y_valid, y_pred_proba_final)
print(f"Final AUC: {auc_final}")

# Plot ROC curve for the final model
fpr_final, tpr_final, thresholds_final = roc_curve(y_valid, y_pred_proba_final)
plt.figure()
plt.plot(fpr_final, tpr_final, color='darkorange', lw=2, label=f'ROC curve (area = {auc_final:0.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Final CNN Model')
plt.legend(loc="lower right")
plt.show()


ModuleNotFoundError: No module named 'tensorflow'